<a href="https://colab.research.google.com/github/w33blaster/Django3-pasword-generator/blob/master/314_How_to_code_the_genetic_algorithm_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://youtu.be/EJeTWRP3Bd0

# What is Genetic algorithm and How to code it in python?<br>
The genetic algorithm is a stochastic method for function optimization inspired by the process of natural evolution - select parents to create children using the crossover and mutation processes.
<p>
The code is an implementation of the genetic algorithm for optimization. The algorithm is used to find the minimum value of a two-dimensional inverted Gaussian function centered at (7,9). The algorithm consists of the following steps:<p>

1. Initialize a population of binary bitstrings with random values.
2. Decode the binary bitstrings into numerical values, and evaluate the fitness (the objective function) for each individual in the population.
3. Select the best individuals from the population using tournament selection based on the fitness scores.
4. Create new offsprings from the selected individuals using the crossover operation.
5. Apply the mutation operation on the offsprings to maintain diversity in the population.
6. Repeat steps 2 to 5 until a stopping criterion is met.

<p>

The implementation includes functions for decoding, selection, crossover, and mutation.

In [1]:
from numpy.random import randint
from numpy.random import rand
import math

In [2]:
# The objective function is a two-dimensional inverted Gaussian function, centred at (7, 9)
def objective(x):
  y = math.exp(((x[0]-7)**2) + (x[1]-9)**2)
  return y

In [28]:
# The decode function decodes binary bitstrings to numbers for each input and scales the value to fall within defined bounds
def decode(bounds, n_bits, bitstring):
	"""
	This function decodes binary bitstrings to numbers for each input and scales the value to fall within defined bounds.

	Parameters:
		bounds (list): A list of tuples representing the lower and upper bounds for each value to be decoded.
		n_bits (int): The number of bits used to represent each value.
		bitstring (list): A binary bitstring to be decoded.

	Returns:
		decoded (list): A list of decoded values.
	"""
	decoded = []  # Create an empty list to hold the decoded values
	largest = 2**n_bits  # The largest value that can be represented with the given number of bits
	for i in range(len(bounds)):
		# Extract the substring for the current value
		start, end = i * n_bits, (i * n_bits) + n_bits  # Define the start and end indices of the substring
		substring = bitstring[start:end]  # Extract the substring
		# Convert the substring to a string of characters
		chars = ''.join([str(s) for s in substring])  # Join the values in the substring together into a string of characters
		# Convert the string of characters to an integer
		integer = int(chars, 2)  # Convert the binary number string into an integer
		# Scale the integer to the desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])  # Scale the integer to a value within the defined bounds
		# Store the decoded value
		decoded.append(value)
	return decoded

In [29]:
# """
###########################################################################
#Let us understand the decoding part first that takes the bounds as input along with the
#number of bits and the actual bitstring itself and decodes the bitstring back
#to original values that fall within the test bounds.
test_bounds=[[-10.0, 10.0], [-10.0, 10.0]]
test_n_bits = 16
#test_n_pop = 100
#Generate a random bit string (of values 0 and 1) of length n_bits*len(bounds). In our case 16*2 = 32
test_bit_string = randint(0, 2, test_n_bits*len(test_bounds)).tolist()
decoded_values = decode(test_bounds, test_n_bits, test_bit_string)
print(test_bit_string)
print(decoded_values)
######################################################################
# """


[1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1]
[4.2425537109375, -8.52386474609375]


In [5]:
def selection(pop, scores, k=3):
    """
    Select the best individuals for the next generation based on their fitness (scores).
    This function randomly selects k individuals from the population and performs a tournament
    among them to choose the one with the best score.

    Parameters:
    pop (list): The population of individuals.
    scores (list): The fitness scores for each individual in the population.
    k (int, optional): The number of individuals to select from the population for the tournament.
                        Defaults to 3.

    Returns:
    individual: The best individual from the tournament.
    """
    # Randomly select one index from the population as the initial selection
    selection_ix = randint(len(pop))
    # Perform a tournament among k randomly selected individuals
    for ix in randint(0, len(pop), k-1):
        # Check if the current individual has a better score than the selected one
        if scores[ix] < scores[selection_ix]:
            # Update the selected individual if a better one is found
            selection_ix = ix
    # Return the best individual from the tournament
    return pop[selection_ix]

In [33]:
# """
#######################################################
#Let us understand the selection process.
#Create a test population
test_bounds=[[-10.0, 10.0], [-10.0, 10.0]]
test_n_bits = 16
test_n_pop = 100
#Create a random population from which we will select based on the scores (from the objective function)
pop = [randint(0, 2, test_n_bits*len(test_bounds)).tolist() for _ in range(test_n_pop)]
# decode population
test_decoded = [decode(test_bounds, test_n_bits, p) for p in pop]
# evaluate all candidates in the population using our objective function
test_scores = [objective(d) for d in test_decoded]
#Run the selection to pick the selected ones from our population
test_selection = selection(pop, test_scores, k=3)

print("From a population of :", len(pop), " the selected pop is: ")
print(test_selection)
################################################
# """

From a population of : 100  the selected pop is: 
[0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1]


In [34]:
def crossover(p1, p2, r_cross):
    """
    Create two children from two parents using the crossover operation.
    The children are created by copying the parents, and recombination occurs
    if a random value is less than the crossover rate.

    Parameters:
    p1 (list): The first parent.
    p2 (list): The second parent.
    r_cross (float): The crossover rate.

    Returns:
    list: A list containing the two children.
    """
    # Children are copies of the parents by default
    c1, c2 = p1.copy(), p2.copy()
    # Check if recombination should occur
    if rand() < r_cross:
        # Select a crossover point (not at the end of the string)
        pt = randint(1, len(p1)-2)
        # Perform crossover in the children
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    # Return the two children
    return [c1, c2]

In [37]:
# """
#######################################################
#Let us understand the Crossover .
test_r_cross = 0.9 #Crossover rate
test_bounds=[[-10.0, 10.0], [-10.0, 10.0]]
test_n_bits = 16
test_p1 = randint(0, 2, test_n_bits*len(test_bounds)).tolist()
test_p2 = randint(0, 2, test_n_bits*len(test_bounds)).tolist()
test_c1, test_c2 = crossover(test_p1, test_p2, test_r_cross)
print("Parent 1 is: ", test_p1)
print("Child 1 is : ", test_c1)
print("Parent 2 is: ", test_p2)
print("Child 2 is : ", test_c2)
#######################################################
# """

Parent 1 is:  [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
Child 1 is :  [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1]
Parent 2 is:  [0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1]
Child 2 is :  [0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]


In [38]:
#The crossover process can generate offsprings that are very similar to the parents. This might cause a new generation with low diversity.
# The mutation process solves this problem by changing the value of some features in the offspring at random.


import random

def mutation(bitstring, r_mut):
    """
    The mutation process changes the value of some features in the offspring at random to maintain the diversity in the population.
    A standard value for the mutation rate is 1/m where m is the number of features.

    Parameters:
    bitstring (list): A list of binary values representing the offspring
    r_mut (float): The mutation rate, typically 1/m where m is the number of features

    Returns:
    list: The modified bitstring after mutation

    """
    rand = random.random
    for i in range(len(bitstring)):
        # Check for a mutation
        if rand() < r_mut:
            # Flip the bit
            bitstring[i] = 1 - bitstring[i]
    return bitstring

In [46]:
# """
#######################################################
#Run the cell a few times to see random mutations
#Let us understand the mutation.
# define range for input
test_bounds = [[-10.0, 10.0], [-10.0, 10.0]]
test_n_bits = 16
# mutation rate
r_mut = 1.0 / (float(test_n_bits) * len(test_bounds))
test_bitstring = randint(0, 2, test_n_bits*len(test_bounds)).tolist()
print("String before mutation is ", test_bitstring)
mutation(test_bitstring, r_mut)
print("String after mutation is  ", test_bitstring)
#################################################
# """

String before mutation is  [0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0]
String after mutation is   [0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0]


In [20]:
### Putting all together into our Genetic algorithm that runs until it finds the best
#The whole fitness assignment, selection, recombination, and mutation process is
#repeated until a stopping criterion is satisfied.
#Each generation is likely to be more adapted to the environment than the old one.

# genetic algorithm implementation
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    """
    The genetic algorithm that finds the optimal solution by performing the fitness assignment, selection, recombination, and mutation process repeatedly.
    Each iteration, the solution is more adapted to the environment.

    Parameters
    ----------
    objective: function
        The objective function that needs to be optimized.
    bounds: list of tuples
        The bounds of the solution.
    n_bits: int
        The number of bits used to encode the solution.
    n_iter: int
        The number of iterations to perform.
    n_pop: int
        The size of the population.
    r_cross: float
        The crossover rate.
    r_mut: float
        The mutation rate.

    Returns
    -------
    list
        The best solution and its evaluation.
    """
    # initialize the population with random bitstrings
    pop = [randint(0, 2, n_bits * len(bounds)).tolist() for _ in range(n_pop)]

    # track the best solution found so far
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))

    # iterate over generations
    for gen in range(n_iter):
        # decode the population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for a new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))

        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]

        # create the next generation - children
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i + 1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace the population
        pop = children
    return [best, best_eval]




In [21]:

# define range for input
bounds = [[-10.0, 10.0], [-10.0, 10.0]]
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('###########################################################')
decoded = decode(bounds, n_bits, best)
print('The result is (%s) with a score of %f' % (decoded, score))

>0, new best f([9.31671142578125, 6.39434814453125]) = 190341.179470
>0, new best f([4.51904296875, 6.9769287109375]) = 28225.052005
>0, new best f([4.32647705078125, 7.91015625]) = 4169.220469
>0, new best f([5.92010498046875, 8.973388671875]) = 3.211960
>3, new best f([6.49627685546875, 8.95843505859375]) = 1.291061
>4, new best f([7.11090087890625, 8.515625]) = 1.280074
>5, new best f([7.1551513671875, 8.80218505859375]) = 1.065243
>5, new best f([7.0458984375, 8.80218505859375]) = 1.042099
>6, new best f([7.14385986328125, 9.03656005859375]) = 1.022277
>6, new best f([6.885986328125, 9.07562255859375]) = 1.018894
>6, new best f([7.12066650390625, 8.958740234375]) = 1.016396
>7, new best f([6.93939208984375, 8.95843505859375]) = 1.005416
>8, new best f([6.93939208984375, 8.9862060546875]) = 1.003871
>8, new best f([6.99249267578125, 8.956298828125]) = 1.001968
>9, new best f([6.9683837890625, 9.022216796875]) = 1.001494
>10, new best f([6.9976806640625, 8.997802734375]) = 1.000010
>